# YOLO Video Detection con Supervision

Este notebook muestra cómo usar YOLO (v8-v11) con la librería **supervision** para:
- Detectar objetos en imágenes y video
- Tracking de objetos (seguimiento entre frames)
- Anotaciones visuales (bounding boxes, etiquetas, trazas)

## 1. Instalación e Imports

In [ ]:
# Instalar dependencias (ejecutar solo la primera vez)
# !pip install ultralytics supervision opencv-python

In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO
from IPython.display import display, Image, Video, clear_output
import matplotlib.pyplot as plt

print(f"Supervision version: {sv.__version__}")

## 2. Cargar el Modelo YOLO

Modelos disponibles (de más rápido a más preciso):
- `yolo11n.pt` - Nano (más rápido)
- `yolo11s.pt` - Small
- `yolo11m.pt` - Medium
- `yolo11l.pt` - Large
- `yolo11x.pt` - Extra Large (más preciso)

También puedes usar `yolov8*.pt`, `yolov9*.pt`, `yolov10*.pt`

In [ ]:
# Cargar modelo (se descarga automáticamente la primera vez)
model = YOLO("yolo11n.pt")

# Ver las clases que puede detectar
print(f"Clases disponibles ({len(model.names)}):")
print(list(model.names.values())[:20], "...")

## 3. Detección en una Imagen

Primero probamos con una imagen estática para entender el flujo.

In [ ]:
# Capturar una imagen de la webcam (o cargar de archivo)
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

if ret:
    # Convertir BGR -> RGB para visualizar
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title("Imagen original")
    plt.axis('off')
    plt.show()
else:
    print("No se pudo capturar imagen. Usando imagen de ejemplo.")
    # Crear imagen de prueba
    frame = np.zeros((480, 640, 3), dtype=np.uint8)

In [ ]:
# Ejecutar detección YOLO
results = model(frame, conf=0.5, verbose=False)[0]

# Convertir a formato supervision
detections = sv.Detections.from_ultralytics(results)

print(f"Objetos detectados: {len(detections)}")
print(f"\nDetalles de detecciones:")
for i, (xyxy, conf, class_id) in enumerate(zip(detections.xyxy, detections.confidence, detections.class_id)):
    print(f"  {i+1}. {model.names[class_id]}: {conf:.2%} - bbox: {xyxy.astype(int)}")

In [ ]:
# Crear anotadores
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.5)

# Generar etiquetas
labels = [
    f"{model.names[class_id]} {conf:.2f}"
    for class_id, conf in zip(detections.class_id, detections.confidence)
]

# Anotar imagen
annotated = frame.copy()
annotated = box_annotator.annotate(annotated, detections=detections)
annotated = label_annotator.annotate(annotated, detections=detections, labels=labels)

# Visualizar
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.title(f"Detección YOLO - {len(detections)} objetos")
plt.axis('off')
plt.show()

## 4. Tracking de Objetos

**ByteTrack** asigna IDs únicos a cada objeto y los mantiene entre frames.

Esto permite:
- Contar objetos únicos
- Seguir trayectorias
- Analizar comportamiento

In [ ]:
# Crear tracker
tracker = sv.ByteTrack()

# Aplicar tracking a las detecciones
detections_tracked = tracker.update_with_detections(detections)

print("Detecciones con tracking:")
if detections_tracked.tracker_id is not None:
    for tid, class_id, conf in zip(detections_tracked.tracker_id, 
                                    detections_tracked.class_id, 
                                    detections_tracked.confidence):
        print(f"  ID #{tid}: {model.names[class_id]} ({conf:.2%})")
else:
    print("  No hay objetos trackeados")

## 5. Procesar Video Completo

Usamos `sv.process_video()` para procesar un archivo de video frame por frame.

In [ ]:
# Configuración
VIDEO_INPUT = "test_video.mp4"   # Cambia por tu video
VIDEO_OUTPUT = "output_notebook.mp4"
CONFIDENCE = 0.5

# Reiniciar tracker para nuevo video
tracker = sv.ByteTrack()

# Crear anotadores
box_annotator = sv.BoxAnnotator(thickness=2)
label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.5)
trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=50)

In [ ]:
def process_frame(frame: np.ndarray, frame_idx: int) -> np.ndarray:
    """
    Callback que procesa cada frame del video.
    
    Args:
        frame: Imagen BGR del frame actual
        frame_idx: Índice del frame (0, 1, 2, ...)
    
    Returns:
        Frame anotado con detecciones
    """
    # 1. Detección YOLO
    results = model(frame, conf=CONFIDENCE, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    
    # 2. Tracking
    detections = tracker.update_with_detections(detections)
    
    # 3. Generar etiquetas con ID de tracking
    if detections.tracker_id is not None:
        labels = [
            f"#{tid} {model.names[cid]} {conf:.2f}"
            for tid, cid, conf in zip(
                detections.tracker_id,
                detections.class_id,
                detections.confidence
            )
        ]
    else:
        labels = [
            f"{model.names[cid]} {conf:.2f}"
            for cid, conf in zip(detections.class_id, detections.confidence)
        ]
    
    # 4. Anotar frame
    annotated = frame.copy()
    annotated = box_annotator.annotate(annotated, detections=detections)
    annotated = label_annotator.annotate(annotated, detections=detections, labels=labels)
    annotated = trace_annotator.annotate(annotated, detections=detections)
    
    # Mostrar progreso cada 50 frames
    if frame_idx % 50 == 0:
        print(f"Frame {frame_idx}...")
    
    return annotated

In [ ]:
# Verificar que existe el video de entrada
import os
if os.path.exists(VIDEO_INPUT):
    # Info del video
    video_info = sv.VideoInfo.from_video_path(VIDEO_INPUT)
    print(f"Video de entrada: {VIDEO_INPUT}")
    print(f"  Resolución: {video_info.width}x{video_info.height}")
    print(f"  FPS: {video_info.fps}")
    print(f"  Total frames: {video_info.total_frames}")
    print(f"  Duración: {video_info.total_frames / video_info.fps:.1f}s")
else:
    print(f"⚠️ No se encontró '{VIDEO_INPUT}'")
    print("Descarga un video de prueba o cambia VIDEO_INPUT")

In [ ]:
# Procesar video (solo si existe)
if os.path.exists(VIDEO_INPUT):
    print(f"\nProcesando video...")
    
    sv.process_video(
        source_path=VIDEO_INPUT,
        target_path=VIDEO_OUTPUT,
        callback=process_frame
    )
    
    print(f"\n✅ Video guardado en: {VIDEO_OUTPUT}")

## 6. Detección en Tiempo Real (Webcam)

⚠️ **Nota**: Este código abre una ventana de OpenCV. En algunos entornos de notebook puede no funcionar correctamente.

In [ ]:
def run_webcam_detection(duration_seconds: int = 30):
    """
    Ejecuta detección en webcam por un tiempo determinado.
    Presiona 'q' para salir antes.
    """
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se puede abrir la webcam")
        return
    
    # Reiniciar tracker
    tracker = sv.ByteTrack()
    
    # Anotadores
    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.5)
    trace_annotator = sv.TraceAnnotator(thickness=2, trace_length=50)
    
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    max_frames = int(duration_seconds * fps)
    frame_count = 0
    
    print(f"Iniciando detección por {duration_seconds}s. Presiona 'q' para salir.")
    
    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detección
        results = model(frame, conf=0.5, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = tracker.update_with_detections(detections)
        
        # Etiquetas
        if detections.tracker_id is not None:
            labels = [f"#{tid} {model.names[cid]}" 
                      for tid, cid in zip(detections.tracker_id, detections.class_id)]
        else:
            labels = [model.names[cid] for cid in detections.class_id]
        
        # Anotar
        annotated = box_annotator.annotate(frame.copy(), detections)
        annotated = label_annotator.annotate(annotated, detections, labels=labels)
        annotated = trace_annotator.annotate(annotated, detections)
        
        # Mostrar
        cv2.imshow("YOLO Webcam", annotated)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Detección finalizada. Frames procesados: {frame_count}")

In [ ]:
# Descomentar para ejecutar webcam (30 segundos)
# run_webcam_detection(30)

## 7. Resumen de la Arquitectura

```
┌─────────────┐     ┌─────────────────┐     ┌─────────────┐     ┌─────────────┐
│   Frame     │ ──► │  YOLO Model     │ ──► │  ByteTrack  │ ──► │  Annotators │
│   (imagen)  │     │  model(frame)   │     │  tracker()  │     │  annotate() │
└─────────────┘     └─────────────────┘     └─────────────┘     └─────────────┘
                            │                      │                    │
                            ▼                      ▼                    ▼
                    ┌───────────────┐      ┌────────────┐       ┌────────────┐
                    │ Detections:   │      │ + tracker_ │       │ Frame con  │
                    │ - xyxy        │      │   id       │       │ boxes,     │
                    │ - confidence  │      │ (IDs       │       │ labels,    │
                    │ - class_id    │      │  únicos)   │       │ traces     │
                    └───────────────┘      └────────────┘       └────────────┘
```

### Clases Principales de Supervision

| Clase | Uso |
|-------|-----|
| `sv.Detections` | Contenedor de detecciones (xyxy, conf, class_id, tracker_id) |
| `sv.ByteTrack` | Algoritmo de tracking multi-objeto |
| `sv.BoxAnnotator` | Dibuja bounding boxes |
| `sv.LabelAnnotator` | Dibuja etiquetas de texto |
| `sv.TraceAnnotator` | Dibuja trazas de movimiento |
| `sv.process_video()` | Procesa video frame por frame |
| `sv.VideoInfo` | Metadatos del video (resolución, fps, frames) |